# User Matching

Notebook author: Martin Saveski (msaveski@mit.edu)

Copyright (c) Facebook, Inc. and its affiliates.

This source code is licensed under the MIT license found in the LICENSE file in the root directory of this source tree.

In [ ]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

Queries for fetching catalyst and control data

In [ ]:
q_catalyst = None
q_control = None

Build indices

In [ ]:
n_posts_user_ids = {}
n_posts_n_friends = {}

df_control_n_posts = q_control['n_posts'].values
df_control_n_friends = q_control['n_friends'].values
df_control_user_ids = q_control['user_id'].values

for i in tqdm(range(len(df_control_user_ids))):
    
    i_n_posts = df_control_n_posts[i]
    i_user_id = df_control_user_ids[i]
    i_n_friends = df_control_n_friends[i]
    
    if np.isnan(df_control_n_friends[i]):
        continue
    
    # add user_id
    n_posts_user_ids_l = n_posts_user_ids.get(i_n_posts, [])
    n_posts_user_ids_l.append(i_user_id)
    n_posts_user_ids[i_n_posts] = n_posts_user_ids_l
    
    # add n_friends
    n_posts_n_friends_l = n_posts_n_friends.get(i_n_posts, [])
    n_posts_n_friends_l.append(i_n_friends)
    n_posts_n_friends[i_n_posts] = n_posts_n_friends_l

Finding users with the same number of posts and closest friend count

In [ ]:
def closest_by_n_friends(cat_n_friends, control_n_friends, control_user_ids, seen):
    
    # [searchsorted] =>
    # right: a[i-1] <= v < a[i]
    # If ‘left’, the index of the first suitable location found is given. 
    # If ‘right’, return the last such index. 
    # If there is no suitable index, return either 0 or N (where N is the length of a).
    
    max_ind = max(len(control_n_friends) - 1, 0)
    m_ind = np.searchsorted(control_n_friends, cat_n_friends, side='left')
    
    if m_ind == len(control_n_friends):
        m_ind = m_ind - 1
    
    if (m_ind - 1) >= 0:
        # check which one is better
        d_m1 = abs(control_n_friends[m_ind - 1] - cat_n_friends)
        dm = abs(control_n_friends[m_ind] - cat_n_friends)
        
        if d_m1 <= dm:
            m_ind = m_ind -1
    
    if control_user_ids[m_ind] not in seen:
        return control_user_ids[m_ind]
    
    else:
        # find an alternative 
        l_ind = m_ind - 1
        r_ind = m_ind + 1
        
        while True:
            if l_ind < 0 and r_ind > max_ind:
                return None
            
            if l_ind < 0:
                l_delta = np.inf
            else:
                l_delta = abs(control_n_friends[l_ind] - cat_n_friends)
                
            if r_ind > max_ind:
                r_delta = np.inf
            else:
                r_delta = abs(control_n_friends[r_ind] - cat_n_friends)
            
            if l_delta < r_delta:
                if control_user_ids[l_ind] not in seen:
                    return control_user_ids[l_ind]
                    
                l_ind -= 1
                
            elif r_delta <= l_delta:
                if control_user_ids[r_ind] not in seen:
                    return control_user_ids[r_ind]
                
                r_ind += 1
    
    return None

In [ ]:
def closest_by_n_posts(cat_n_posts, cat_n_friends, n_posts_n_friends, n_posts_user_ids, seen):
    
    if cat_n_posts not in n_posts_user_ids:
        return None
        
    m_user_id = closest_by_n_friends(
        cat_n_friends, 
        n_posts_n_friends[cat_n_posts], 
        n_posts_user_ids[cat_n_posts], 
        seen
    )
    
    if m_user_id is not None:
        seen.add(m_user_id)
        
    return m_user_id

In [ ]:
max_d_posts = 5

df_catalyst_n_posts = q_catalyst['n_posts'].values
df_catalyst_n_friends = q_catalyst['n_friends'].values
df_catalyst_user_ids = q_catalyst['user_id'].values
n_catalysts = len(df_catalyst_n_posts)

seen = set()
matching = {} # catalyst_user_id => control_user_id

d_posts = []
n_not_matched = 0

for i in tqdm(range(n_catalysts)):
    if i % 100000 == 0: 
        print(f"#{i} | # not matched: {n_not_matched} | # seen {len(seen)}")
    
    cat_n_posts = df_catalyst_n_posts[i]
    cat_user_id = df_catalyst_user_ids[i]
    cat_n_friends = df_catalyst_n_friends[i]
    
    m_user_id = None
    sgns = random.sample([-1, 1], 2)
    steps = [0] + [sgn * i for i in range(1, max_d_posts + 1) for sgn in sgns]
    
    for step in steps:
        cat_n_posts_step = cat_n_posts + step
        
        m_user_id = closest_by_n_posts(
            cat_n_posts_step, 
            cat_n_friends, 
            n_posts_n_friends, 
            n_posts_user_ids, 
            seen
        )
        
        if m_user_id is not None:
            break
    
    if m_user_id is not None:
        matching[cat_user_id] = m_user_id
        d_posts.append(abs(step))
    
    else:
        print("match not found", cat_n_posts)
        n_not_matched += 1

Output

In [ ]:
df_out = pd.DataFrame(
    data={
        "catalyst_user_id": list(matching.keys()), 
        "matched_user_id": list(matching.values())
    })